In [1]:
!pip install crewai
!pip install duckduckgo-search
!pip install langchain_community
#export GROQ_API_KEY=<your-api-key-here>

In [2]:
from langchain_community.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os
from langchain_groq import ChatGroq

In [3]:
#api_key= os.environ.get("$GROQ_API_KEY")
#model_name="llama-3.1-70b-versatile"

#llm = ChatGroq(
#    model=model_name,
#    temperature=0.7,
#    max_tokens=1024,
#    timeout=None,
#    max_retries=2
#)

In [4]:
search_tool = DuckDuckGoSearchRun()

In [5]:
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOpenAI(
    model = "llama3.1",
    base_url = "http://192.168.1.124:11434/v1",
    temperature=0.7,
    max_tokens=1024,
    timeout=None,
    max_retries=2
)

In [6]:
inputs = {
    "topic":"Step by step example of installing Apache Ignite on Docker.",
    "target_audience": "Programmer, Product owner, Analyst"
}

In [7]:
#Content planner Agent
planner = Agent(
    role="Content Planner",
    goal="You are tasked with planning an engaging and informative blog post on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}. "
              "Your goal is to gather accurate, up-to-date information and structure the content " 
              "in a way that captures the reader’s interest while delivering valuable insights. "
              "The audience for this blog post is {target_audience}, "
              "and they are looking for clear, practical information that they can apply immediately. "
              "You must consider their needs, interests, and potential challenges they might face related to the topic. "
              "Your job is to create an outline that will guide the writer in crafting a piece that "
              "is both educational and easy to digest. "
              "Your work is the base for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [8]:
#Content writer Agent
writer = Agent(
  role='Technical content writer',
  goal="You are the technical content writer assigned to create a detailed and factually accurate blog post on: {topic}",
  backstory="You're working on a writing "
              "a new step by step tutorial about the topic: {topic}. "
              "Your job is to ensure that the content is factually accurate, well-organized, "
              "and clearly explains the topic at hand. You will need to break down complex "
              "technical concepts into simple, digestible parts, while still "
              "maintaining depth and technical rigor. You must also incorporate "
              "relevant data, examples, and use cases to make the article practical and valuable to the reader. "  
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "Your audience consists of {target_audience}, "
              "who are likely seeking clear and actionable insights. "
              "Whether it's professionals looking to apply the knowledge "
              "in their work, or beginners trying to grasp the basics, "
              "your writing should meet their needs. Make sure to maintain an "
              "approachable tone, balancing technical depth with readability.",
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [9]:
editor = Agent(
    role="Editor",
    goal="You responsible for reviewing and refining a blog post to ensure it aligns with the clear, "
    "concise, and engaging writing style of platforms like. 'https://dzone.com'. ",
    backstory="You are the technical content editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to make sure the post is grammatically correct, "
              "free of errors, and easy to read. You will need to carefully "
              "check sentence structure, punctuation, and overall coherence "
              "while ensuring the tone remains conversational, yet informative—similar "
              "to the approachable style used on Medium.com. The content should feel "
              "natural, guiding readers effortlessly from one section to the next, "
              "while maintaining technical accuracy.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
# Create tasks for the planner agent
plan = Task(
    description=(
        "1. Step by step example , "
            "and noteworthy latest news on {topic}.\n"
        "2. Identify the {target_audience}, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, values, key points, and step by step example.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    tools=[search_tool],
)

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 3 or 5 paragraphs.",
    agent=writer,
)

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with human language for better readability while maintaining its original meaning."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 3 or 5 paragraphs.",
    agent=editor
)

In [13]:
# Instantiate the crew with a sequential process
crew = Crew(
  agents=[planner, writer, editor],
  tasks=[plan, edit, write],
  verbose=2, # You can set it to 1 or 2 to different logging levels
) 

In [14]:
# run the crew
result = crew.kickoff(inputs=inputs)

print("**Final result**")
print(result)

 [2024-09-27 13:55:54][DEBUG]: == Working Agent: Content Planner
 [2024-09-27 13:55:54][INFO]: == Starting Task: 1. Step by step example , and noteworthy latest news on Step by step example of installing Apache Ignite on Docker..
2. Identify the Programmer, Product owner, Analyst, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, values, key points, and step by step example.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: To create a comprehensive content plan for the blog post "Step by step example of installing Apache Ignite on Docker", I need to start by gathering information on the latest news and updates on Apache Ignite and Docker.

Action: duckduckgo_search
Action Input: {"query": "Apache Ignite Docker latest news"} 

The new Apache Ignite 2.11.1 is an emergency release that fixes CVE-2021-44228, CVE-2021-45046,CVE-2021-45105 related to the ignite-log4j2 module 